In [3]:
import pandas as pd
import json
df1 = pd.read_csv('../data/data.csv',  encoding='cp949')

In [4]:
df2 = pd.read_csv('../data/서울시 상권분석서비스(영역-상권).csv',  encoding='cp949')

In [8]:
df1_unique = df1['상권_코드_명'].unique()
df2_unique = df2['상권_코드_명'].unique()
len(set(df1_unique) - set(df2_unique))


1

In [10]:
print(len(df1_unique), len(df2_unique))

1564 1650


In [9]:
set(df1_unique) - set(df2_unique)
# 종로 청계 관광특구만 따로 위도 경도를 구해주자

{'종로·청계 관광특구'}

In [15]:
df1[df1['상권_코드_명'] == '종로·청계 관광특구'].head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,...,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
24,20221,U,관광특구,3001494,종로·청계 관광특구,CS200024,스포츠클럽,3.448275e+08,745,3.387755e+08,...,433,8,404,341,0,124,248,187,178,8
506,20221,U,관광특구,3001494,종로·청계 관광특구,CS300018,의약품,9.113443e+09,257070,7.151099e+09,...,53290,2685,137990,115990,389,24188,35735,38028,51748,103888
629,20221,U,관광특구,3001494,종로·청계 관광특구,CS300028,화초,7.241075e+07,717,7.143046e+07,...,211,0,224,221,0,36,168,99,74,63
671,20221,U,관광특구,3001494,종로·청계 관광특구,CS300036,조명용품,4.668823e+10,171883,4.584790e+10,...,10586,0,98699,17484,0,2205,7833,25293,40278,40571
914,20221,U,관광특구,3001494,종로·청계 관광특구,CS100006,패스트푸드점,2.078662e+08,27307,1.582506e+08,...,7828,1954,14729,11400,360,8496,7365,4054,3664,2190


In [18]:
df2.loc[df2['상권_코드'] == 3001494, '상권_코드_명'] = '종로·청계 관광특구'

In [19]:
df2[df2['상권_코드'] == 3001494]['상권_코드_명']

1647    종로·청계 관광특구
Name: 상권_코드_명, dtype: object

In [21]:
df1_unique = df1['상권_코드_명'].unique()
df2_unique = df2['상권_코드_명'].unique()

set(df1_unique) - set(df2_unique)

set()

# 위도 경도로 변환하기

In [24]:
from pyproj import Proj, transform

# EPSG:5181 (Korea 2000 / Central Belt) 좌표계 정의
proj_5181 = Proj(init='epsg:5181')

# WGS84 좌표계 정의 (경도, 위도)
proj_wgs84 = Proj(init='epsg:4326')

# 변환할 좌표 예시 (x, y 형식)
x_5181, y_5181 = 200507, 453618 

# 좌표 변환 수행
lon, lat = transform(proj_5181, proj_wgs84, x_5181, y_5181)

print(f"Converted Longitude: {lon}, Latitude: {lat}")


Converted Longitude: 127.00573993541379, Latitude: 37.58211587382123


/Users/yunju/miniforge3/envs/cg/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/yunju/miniforge3/envs/cg/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/d6/d15px6nn599bb66tcmrq23380000gn/T/ipykernel_80965/2324402055.py:13: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#u

In [23]:
lon, lat

(126.9670895683088, 37.58030943835253)

실제로 확인해본 결과 아주 살짝 정확도가 어긋난다. 500m정도... 그래서 카카오 api를 통해서 변환하는게 나을 듯 하다

# ? 데이터 바꾸기 

In [25]:
df2['행정동_코드_명'] = df2['행정동_코드_명'].replace('?', '·')


In [26]:
df2

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적
0,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,종로구,11110515,청운효자동,149264
1,A,골목상권,3110009,자하문터널,196991,455057,11110,종로구,11110550,부암동,178306
2,A,골목상권,3110010,평창동서측,197064,456643,11110,종로구,11110560,평창동,369415
3,A,골목상권,3110017,정독도서관,198581,453781,11110,종로구,11110600,가회동,83855
4,A,골목상권,3110018,중앙고등학교,198883,453690,11110,종로구,11110600,가회동,166872
...,...,...,...,...,...,...,...,...,...,...,...
1645,U,관광특구,3001492,명동 남대문 북창동 다동 무교동 관광특구,198397,451614,11140,중구,11140520,소공동,983618
1646,U,관광특구,3001493,동대문패션타운 관광특구,200996,451976,11140,중구,11140590,광희동,606058
1647,U,관광특구,3001494,종로·청계 관광특구,199796,452274,11110,종로구,11110615,종로1?2?3?4가동,653127
1648,U,관광특구,3001495,잠실 관광특구,210188,446334,11710,송파구,11710562,방이2동,2462734


In [ ]:
# 안바뀌네.. 근데 어쳐피 경도 위도로 바꾸는 것이고 구단위로 필터링할 것이기에 굳이 바꾸지는 않아도 될 듯
# 그리고 생각해보니 뭐시기 관광특구는 카카오 api에 인식이 안된다. 그냥 pyproj를 사용하자

# 경도위도로 바꾸기

In [27]:
df1_unique = df1['상권_코드_명'].unique()
df2_unique = df2['상권_코드_명'].unique()

set(df1_unique) - set(df2_unique)

set()

In [28]:
df2_filtered = df2[df2['상권_코드_명'].isin(df1_unique)]


In [31]:
len(df1_unique)

1564

In [29]:
df2_filtered

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적
0,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,종로구,11110515,청운효자동,149264
1,A,골목상권,3110009,자하문터널,196991,455057,11110,종로구,11110550,부암동,178306
2,A,골목상권,3110010,평창동서측,197064,456643,11110,종로구,11110560,평창동,369415
3,A,골목상권,3110017,정독도서관,198581,453781,11110,종로구,11110600,가회동,83855
4,A,골목상권,3110018,중앙고등학교,198883,453690,11110,종로구,11110600,가회동,166872
...,...,...,...,...,...,...,...,...,...,...,...
1645,U,관광특구,3001492,명동 남대문 북창동 다동 무교동 관광특구,198397,451614,11140,중구,11140520,소공동,983618
1646,U,관광특구,3001493,동대문패션타운 관광특구,200996,451976,11140,중구,11140590,광희동,606058
1647,U,관광특구,3001494,종로·청계 관광특구,199796,452274,11110,종로구,11110615,종로1?2?3?4가동,653127
1648,U,관광특구,3001495,잠실 관광특구,210188,446334,11710,송파구,11710562,방이2동,2462734


In [32]:
def convert_coordinates(row):
    proj_5181 = Proj(init='epsg:5181')
    proj_wgs84 = Proj(init='epsg:4326')
    # 데이터프레임의 행에서 x, y 좌표 추출
    x_5181, y_5181 = row['엑스좌표_값'], row['와이좌표_값']
    # 좌표 변환 수행
    lon, lat = transform(proj_5181, proj_wgs84, x_5181, y_5181)
    return pd.Series([lon, lat])


In [33]:
df2_filtered[['경도', '위도']] = df2_filtered.apply(convert_coordinates, axis=1)


/Users/yunju/miniforge3/envs/cg/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/yunju/miniforge3/envs/cg/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/d6/d15px6nn599bb66tcmrq23380000gn/T/ipykernel_80965/919936350.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upg

In [36]:
df2_filtered

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적,경도,위도
0,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,종로구,11110515,청운효자동,149264,126.967090,37.580309
1,A,골목상권,3110009,자하문터널,196991,455057,11110,종로구,11110550,부암동,178306,126.965928,37.595076
2,A,골목상권,3110010,평창동서측,197064,456643,11110,종로구,11110560,평창동,369415,126.966748,37.609366
3,A,골목상권,3110017,정독도서관,198581,453781,11110,종로구,11110600,가회동,83855,126.983935,37.583584
4,A,골목상권,3110018,중앙고등학교,198883,453690,11110,종로구,11110600,가회동,166872,126.987354,37.582764
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645,U,관광특구,3001492,명동 남대문 북창동 다동 무교동 관광특구,198397,451614,11140,중구,11140520,소공동,983618,126.981856,37.564059
1646,U,관광특구,3001493,동대문패션타운 관광특구,200996,451976,11140,중구,11140590,광희동,606058,127.011274,37.567321
1647,U,관광특구,3001494,종로·청계 관광특구,199796,452274,11110,종로구,11110615,종로1?2?3?4가동,653127,126.997691,37.570007
1648,U,관광특구,3001495,잠실 관광특구,210188,446334,11710,송파구,11710562,방이2동,2462734,127.115241,37.516431


In [38]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83573 entries, 0 to 83572
Data columns (total 55 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준_년분기_코드        83573 non-null  int64  
 1   상권_구분_코드         83573 non-null  object 
 2   상권_구분_코드_명       83573 non-null  object 
 3   상권_코드            83573 non-null  int64  
 4   상권_코드_명          83573 non-null  object 
 5   서비스_업종_코드        83573 non-null  object 
 6   서비스_업종_코드_명      83573 non-null  object 
 7   당월_매출_금액         83573 non-null  float64
 8   당월_매출_건수         83573 non-null  int64  
 9   주중_매출_금액         83573 non-null  float64
 10  주말_매출_금액         83573 non-null  float64
 11  월요일_매출_금액        83573 non-null  float64
 12  화요일_매출_금액        83573 non-null  float64
 13  수요일_매출_금액        83573 non-null  float64
 14  목요일_매출_금액        83573 non-null  float64
 15  금요일_매출_금액        83573 non-null  float64
 16  토요일_매출_금액        83573 non-null  float64
 17  일요일_매출_금액   

# data에 붙이기

In [39]:
merged_df = pd.merge(df1, df2_filtered, on='상권_코드_명', how='inner', suffixes=('', '_from_df2'))

In [41]:
merged_df.columns

Index(['기준_년분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드',
       '서비스_업종_코드_명', '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액',
       '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액',
       '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액',
       '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액',
       '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액',
       '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액',
       '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수',
       '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수',
       '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수',
       '시간대_건수~17_매출_건수', '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수',
       '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수',
       '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '상권_구분_코드_from_df2',
       '상권_구분_코드_명_from_df2', '상권_코드_from_df2', '엑스좌표_값', '와이좌표_값', '자치구_

In [43]:
merged_df.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,...,상권_코드_from_df2,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적,경도,위도
0,20221,D,발달상권,3120068,사가정역,CS200016,당구장,6.207432e+06,362,3619902.0,...,3120068,207784,453510,11260,중랑구,11260575,면목3?8동,35712,127.088124,37.58111
1,20221,D,발달상권,3120068,사가정역,CS200024,스포츠클럽,6.653658e+07,79,59686948.0,...,3120068,207784,453510,11260,중랑구,11260575,면목3?8동,35712,127.088124,37.58111
2,20221,D,발달상권,3120068,사가정역,CS100003,일식음식점,2.938315e+07,537,22193773.0,...,3120068,207784,453510,11260,중랑구,11260575,면목3?8동,35712,127.088124,37.58111
3,20221,D,발달상권,3120068,사가정역,CS100010,커피-음료,5.514100e+08,98381,362815465.0,...,3120068,207784,453510,11260,중랑구,11260575,면목3?8동,35712,127.088124,37.58111
4,20221,D,발달상권,3120068,사가정역,CS100005,제과점,1.089305e+09,107851,778947221.0,...,3120068,207784,453510,11260,중랑구,11260575,면목3?8동,35712,127.088124,37.58111


In [44]:
merged_df.drop(columns=['상권_구분_코드_from_df2', '상권_구분_코드_명_from_df2', '상권_코드_from_df2', '엑스좌표_값', '와이좌표_값', '자치구_코드', '행정동_코드', '영역_면적'], inplace=True)


In [45]:
merged_df.columns


Index(['기준_년분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드',
       '서비스_업종_코드_명', '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액',
       '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액',
       '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액',
       '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액',
       '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액',
       '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액',
       '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수',
       '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수',
       '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수',
       '시간대_건수~17_매출_건수', '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수',
       '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수',
       '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '자치구_코드_명',
       '행정동_코드_명', '경도', '위도'],
      dtype='object')

In [46]:
merged_df.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,...,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,자치구_코드_명,행정동_코드_명,경도,위도
0,20221,D,발달상권,3120068,사가정역,CS200016,당구장,6.207432e+06,362,3619902.0,...,0,42,38,45,150,87,중랑구,면목3?8동,127.088124,37.58111
1,20221,D,발달상권,3120068,사가정역,CS200024,스포츠클럽,6.653658e+07,79,59686948.0,...,0,35,36,4,4,0,중랑구,면목3?8동,127.088124,37.58111
2,20221,D,발달상권,3120068,사가정역,CS100003,일식음식점,2.938315e+07,537,22193773.0,...,0,232,194,79,21,11,중랑구,면목3?8동,127.088124,37.58111
3,20221,D,발달상권,3120068,사가정역,CS100010,커피-음료,5.514100e+08,98381,362815465.0,...,2483,31509,29824,16211,12332,5967,중랑구,면목3?8동,127.088124,37.58111
4,20221,D,발달상권,3120068,사가정역,CS100005,제과점,1.089305e+09,107851,778947221.0,...,1890,19462,26284,23903,20723,13994,중랑구,면목3?8동,127.088124,37.58111


In [47]:
merged_df.to_csv('data_xy.csv')

# 다른 json 데이터 확인해보기

In [ ]:
data/서울시 상권분석서비스(추정매출-상권).json

In [48]:
with open('../data/서울시 상권분석서비스(추정매출-상권).json', 'r') as file:
    data = json.load(file)

In [51]:
len(data)

2

In [54]:
data.keys()

dict_keys(['DESCRIPTION', 'DATA'])

In [57]:
len(data['DESCRIPTION'])

55

In [58]:
len(data['DATA'])

62856

In [61]:
data['DATA'][0].keys()

dict_keys(['tmzon_00_06_selng_amt', 'sat_selng_amt', 'ml_selng_co', 'agrde_10_selng_co', 'agrde_40_selng_amt', 'agrde_40_selng_co', 'agrde_20_selng_co', 'thur_selng_amt', 'agrde_10_selng_amt', 'agrde_20_selng_amt', 'fml_selng_co', 'sun_selng_amt', 'svc_induty_cd_nm', 'wed_selng_co', 'fml_selng_amt', 'svc_induty_cd', 'tmzon_17_21_selng_amt', 'agrde_30_selng_amt', 'agrde_60_above_selng_amt', 'agrde_50_selng_co', 'mdwk_selng_amt', 'thsmon_selng_amt', 'tmzon_06_11_selng_co', 'tmzon_00_06_selng_co', 'tmzon_06_11_selng_amt', 'sat_selng_co', 'thsmon_selng_co', 'mon_selng_co', 'fri_selng_co', 'ml_selng_amt', 'tmzon_14_17_selng_amt', 'agrde_60_above_selng_co', 'tmzon_14_17_selng_co', 'fri_selng_amt', 'sun_selng_co', 'agrde_50_selng_amt', 'wkend_selng_co', 'tmzon_11_14_selng_amt', 'stdr_yyqu_cd', 'trdar_se_cd_nm', 'mdwk_selng_co', 'wkend_selng_amt', 'tmzon_21_24_selng_amt', 'tmzon_21_24_selng_co', 'tues_selng_amt', 'trdar_cd', 'mon_selng_amt', 'trdar_se_cd', 'tues_selng_co', 'trdar_cd_nm', 'thur